In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
import quadratic_weighted_kappa
import feature_generator
from scipy import optimize
from NN import NN
from XgBoost import XGBoostModel
from CutPoints import CutPointOptimizer

Using Theano backend.


/Library/Python/2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [4]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')  

train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 10000, True)

featureImpDf = pd.read_csv('FeatureImportance.csv')
importantFeatures = featureImpDf[featureImpDf['Importance'] > 11]
importantFeatures = [column for column in train.columns if column in importantFeatures['Feature'].values]
importantFeatures = importantFeatures['Feature'].values
train = train[importantFeatures]
test = test[importantFeatures]

dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')  

Scaling...


TypeError: list indices must be integers, not str

In [ ]:
model = XGBoostModel(900, 7, 0.025, 0.50, 25)
model.fit(train, labels, 0)

In [ ]:
importance = model.bst.get_fscore()

In [ ]:
importance = sorted(importance.items(), key=operator.itemgetter(1))
for item in importance:
    print item

imp = pd.DataFrame()
imp['Feature'] = [item[0] for item in importance]
imp['Importance'] = [item[1] for item in importance]
imp.to_csv(path_or_buf='FeatureImportance.csv', index=False)

In [5]:
class CombinedModel:
    
    def __init__(self, modelsFromFile, modelsToCalculate):
        self.modelsToCalculate = modelsToCalculate 
        self.modelsFromFile = modelsFromFile
        
    def fit(self, X, Y, fileName, num):     
        
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))    
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            model.fit(X, Y, num)
            predictions = model.predict(X)
            stackingData[:,i] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
        self.stackingModel.fit(stackingData, Y)
        predictions = np.mean(stackingData, axis=1)
        
        initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])
        cpo = CutPointOptimizer(predictions, Y)
        self.cutPoints = optimize.fmin(cpo.qwk, initialCutPoints)
        
        predictions = np.searchsorted(self.cutPoints, predictions) + 1   

        trainQwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)
        print "Train QWK: %s\n" % trainQwk
                           
    def predict(self, X, fileName, num):
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            predictions = model.predict(X)
            stackingData[:,i] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
        predictions = np.mean(stackingData, axis=1)
        return np.searchsorted(self.cutPoints, predictions) + 1         

In [1]:
from keras import objectives

X_train = train
y_train = labels

folds = 8
kf = KFold(len(X_train), folds)
num = 1
combinedModels = list()
qwks = list()

nnModel = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [250, 75], dropout = [0.5, 0.5], activation='relu', patience=5, loss=custom_loss, optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)
xgBoostModel = XGBoostModel(700, 7, 0.025, 0.50, 25)
for train_index, test_index in kf:
    
    combinedModel = CombinedModel([], [nnModel, xgBoostModel])
    
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)

    xTrain = X_train.iloc[train_index].values
    yTrain = y_train.iloc[train_index]      
    xValidate = X_train.iloc[test_index].values
    yValidate = y_train.iloc[test_index]

    combinedModel.fit(xTrain, yTrain, trainFile, num-1)
    predictions = combinedModel.predict(xValidate, validateFile, num-1)
    qwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
    qwks.append(qwk)
    print qwk
    
    combinedModels.append(combinedModel)
    num += 1
    
meanQwk = quadratic_weighted_kappa.mean_quadratic_weighted_kappa(qwks)
print "Overall Test Qwk: %s" % meanQwk

Using Theano backend.


/Library/Python/2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


NameError: name 'train' is not defined

In [6]:
testPredictions = np.zeros(len(test))
for i in range(1, folds + 1):
    model = combinedModels[i-1]
    testPredictions += model.predict(test, 'combinedTestPredictions%s.csv' % str(i), i)

testPredictions /= folds
predDf = pd.DataFrame()
predDf['Id'] = dfTest['Id']
predDf['Response'] = np.round(testPredictions).astype(int)
print predDf['Response'].values
predDf.to_csv(path_or_buf='XgBoost.csv', columns=['Id', 'Response'], index=False, header=['Id', 'Response'])

NameError: name 'folds' is not defined

In [ ]:
kf = KFold(len(train), folds)
num = 1
qwks = list()

name = 'Boost'
# model = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [250, 75, 15], dropout = [0.5, 0.5, 0.5], activation='relu', patience=5, loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)

model = XGBoostModel(800, 7, 0.025, 0.50, 25)

for train_index, test_index in kf:
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)
    testFile = 'combinedTestPredictions%s.csv' % str(num)
    
    trainDF = pd.read_csv(trainFile) if os.path.isfile(trainFile) else pd.DataFrame()  
    validateDF = pd.read_csv(validateFile) if os.path.isfile(validateFile) else pd.DataFrame()  
    testDF = pd.read_csv(testFile) if os.path.isfile(testFile) else pd.DataFrame()  

    xTrain = train.iloc[train_index].values
    yTrain = labels.iloc[train_index]      
    xValidate = train.iloc[test_index].values
    yValidate = labels.iloc[test_index]
    
    model.fit(xTrain, yTrain, num-1)
    testPredictions = model.predict(test.values)
    trainPredictions = model.predict(xTrain)
    validatePredictions = model.predict(xValidate)
    
    trainDF[name] = trainPredictions
    validateDF[name] = validatePredictions
    testDF[name] = testPredictions
        
    trainDF.to_csv(path_or_buf=trainFile, index=False)
    validateDF.to_csv(path_or_buf=validateFile, index=False)
    testDF.to_csv(path_or_buf=testFile, index=False)
            
    num += 1